# Import

In [1]:
import numpy as np
from timeit import default_timer as time
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from numpy.linalg import inv
import functions_eq_mag as fc
import functions_fatiando as ft
import fourier_continuation as fc_c
import pickle as pickle
%matplotlib inline

# Observation and Equivalent layer grids

In [82]:
# Create a regular grid at 0m height
area = [-5000., 5000., -4000., 4000.]
shape = (200, 100)
N = shape[0]*shape[1]
xi, yi, zi = ft.gridder_regular(area, shape, z=-900.)
xi_plot, yi_plot, = xi/1000, yi/1000

# Equivalent Layer
areaj = [-5000., 5000., -4000., 4000.]
shapej = (200, 100)
Nj = shapej[0]*shapej[1]
xj, yj, zj = ft.gridder_regular(areaj, shapej, z=50)

#: Conversion factor from SI units to mGal: :math:`1\ m/s^2 = 10^5\ mGal`
SI2MGAL = 100000.0
#: The gravitational constant in :math:`m^3 kg^{-1} s^{-1}`
G = 0.00000000006673

# Magnetic Configuration
inc0 = np.deg2rad(35.26)
dec0 = np.deg2rad(45.)
inc = np.deg2rad(35.26)
dec = np.deg2rad(45.)

F = np.array([np.cos(inc0)*np.cos(dec0), np.cos(inc0)*np.sin(dec0), np.sin(inc0)])
h = np.array([np.cos(inc)*np.cos(dec), np.cos(inc)*np.sin(dec), np.sin(inc)])

In [83]:
def bttb(x,y,z,h):
    '''
    Calculates the first row of sensbility matrix.

    input
    x, y: numpy array - the x, y coordinates
    of the grid and equivalent layer points.
    z: numpy array - the height of observation points.
    h: numpy array - the depth of the equivalent layer.

    output
    W_bt: numpy array - first line os sensibility matrix.
    '''
    a = (x-x[0])
    b = (y-y[0])
    c = (h-z[0])
    W_bt = (G*SI2MGAL*c)/((a*a+b*b+c*c)**(1.5))
    return W_bt

In [84]:
def cev_grav(shape,N,BTTB):
    '''
    Calculates the eigenvalues of the BCCB matrix.

    input
    shape: tuple - grid size.
    N: scalar - number of observation points.
    BTTB: numpy array - first line os sensibility matrix.

    output
    cev: numpy array - eigenvalues of the BCCB matrix.
    '''
    cev = np.zeros(4*N)
    k = 2*shape[0]-1
    for i in range (shape[0]):
        block = BTTB[shape[1]*(i):shape[1]*(i+1)]
        rev = block[::-1]
        cev[shape[1]*(2*i):shape[1]*(2*i+2)] = np.concatenate((block,0,rev[:-1]), axis=None)
        if i > 0:
            cev[shape[1]*(2*k):shape[1]*(2*k+2)] = cev[shape[1]*(2*i):shape[1]*(2*i+2)]
            k -= 1
    cev = cev.reshape(2*shape[0],2*shape[1])
    return cev

In [85]:
def h_bttb_mag(x,y,z,zj,F,h,shape):
    '''
    Calculates the first row of the sensitivity matrix

    input
    x, y: numpy array - the x, y coordinates
    of the grid and equivalent layer points.
    z: numpy array - the height of observation points.
    h: numpy array - the depth of the equivalent layer.
    shape: tuple - grid size.
    potential field at the grid points.
    F: numpy array - cosines directions of the main magnetic field.
    h: numpy array - cosines directions of the body's magnetization.

    output
    bttb_0: numpy array - first line of the first row of blocks of the
    sensitivity matrix.
    bttb_1: numpy array - last line of the first row of blocks of the
    sensitivity matrix.
    bttb_2: numpy array - first line of the last row of blocks of the
    sensitivity matrix.
    bttb_3: numpy array - last line of the last row of blocks of the
    sensitivity matrix.
    '''
    # First row of each component of the second derivatives
    a = (x-x[0])
    b = (y-y[0])
    c = (z-zj[0])
    r = (a*a+b*b+c*c)
    r3 = r**(-1.5)
    r5 = r**(2.5)
    Hxx = -r3+3*(a*a)/r5
    Hxy = 3*(a*b)/r5
    Hxz = 3*(a*c)/r5
    Hyy = -r3+3*(b*b)/r5
    Hyz = 3*(b*c)/r5
    Hzz = -r3+3*(c*c)/r5

    return Hxx,Hxy,Hxz,Hyy,Hyz,Hzz

In [86]:
def ones_cev_mag(Hxx,Hxy,Hxz,Hyy,Hyz,Hzz,shape,N,F,h):
    '''
    Calculates the eigenvalues of the BCCB matrix using 
    only the effect of one equivalent source.

    input
    shape: tuple - grid size.
    N: scalar - number of observation points.
    Hxx,Hxy,Hxz,Hyy,Hyz,Hzz: numpy array - first row of each component of 
    the second derivatives of 1/r necessary to calculate the BCCB eigenvalues.

    output
    cev: numpy array - eigenvalues of the BCCB matrix.
    '''
    bccb_xx = np.zeros(4*N)
    bccb_xy = np.zeros(4*N)
    bccb_xz = np.zeros(4*N)
    bccb_yy = np.zeros(4*N)
    bccb_yz = np.zeros(4*N)
    bccb_zz = np.zeros(4*N)
    k = 2*shape[0]-1

    for i in range (shape[0]):
        block_xx = Hxx[shape[1]*(i):shape[1]*(i+1)]
        block_xy = Hxy[shape[1]*(i):shape[1]*(i+1)]
        block_xz = Hxz[shape[1]*(i):shape[1]*(i+1)]
        block_yy = Hyy[shape[1]*(i):shape[1]*(i+1)]
        block_yz = Hyz[shape[1]*(i):shape[1]*(i+1)]
        block_zz = Hzz[shape[1]*(i):shape[1]*(i+1)]
        rev_xx = block_xx[::-1]
        rev_xy = -block_xy[::-1]
        rev_xz = block_xz[::-1]
        rev_yy = block_yy[::-1]
        rev_yz = -block_yz[::-1]
        rev_zz = block_zz[::-1]
        bccb_xx[shape[1]*(2*i):shape[1]*(2*i+2)] = np.concatenate((block_xx,
        0,rev_xx[:-1]), axis=None)
        bccb_xy[shape[1]*(2*i):shape[1]*(2*i+2)] = np.concatenate((block_xy,
        0,rev_xy[:-1]), axis=None)
        bccb_xz[shape[1]*(2*i):shape[1]*(2*i+2)] = np.concatenate((block_xz,
        0,rev_xz[:-1]), axis=None)
        bccb_yy[shape[1]*(2*i):shape[1]*(2*i+2)] = np.concatenate((block_yy,
        0,rev_yy[:-1]), axis=None)
        bccb_yz[shape[1]*(2*i):shape[1]*(2*i+2)] = np.concatenate((block_yz,
        0,rev_yz[:-1]), axis=None)
        bccb_zz[shape[1]*(2*i):shape[1]*(2*i+2)] = np.concatenate((block_zz,
        0,rev_zz[:-1]), axis=None)
        if i > 0:
            bccb_xx[shape[1]*(2*k):shape[1]*(2*k+2)] = bccb_xx[shape[1]*
            (2*i):shape[1]*(2*i+2)]
            bccb_xy[shape[1]*(2*k):shape[1]*(2*k+2)] = -bccb_xy[shape[1]*
            (2*i):shape[1]*(2*i+2)]
            bccb_xz[shape[1]*(2*k):shape[1]*(2*k+2)] = -bccb_xz[shape[1]*
            (2*i):shape[1]*(2*i+2)]
            bccb_yy[shape[1]*(2*k):shape[1]*(2*k+2)] = bccb_yy[shape[1]*
            (2*i):shape[1]*(2*i+2)]
            bccb_yz[shape[1]*(2*k):shape[1]*(2*k+2)] = bccb_yz[shape[1]*
            (2*i):shape[1]*(2*i+2)]
            bccb_zz[shape[1]*(2*k):shape[1]*(2*k+2)] = bccb_zz[shape[1]*
            (2*i):shape[1]*(2*i+2)]
            k -= 1

    bccb = 100*((F[0]*bccb_xx+F[1]*bccb_xy+F[2]*bccb_xz)*h[0] + (F[0]*
    bccb_xy+F[1]*bccb_yy+F[2]*bccb_yz)*h[1] + (F[0]*bccb_xz+F[1]*
    bccb_yz+F[2]*bccb_zz)*h[2])

    BCCB = bccb.reshape(2*shape[0],2*shape[1])
    return BCCB

### BCCB  Grav

In [87]:
BTTB_grav1 = bttb(xi,yi,zi,zj)

In [88]:
s = time()
bccb_grav = cev_grav(shape,N,BTTB_grav1)
w = time()
print(w-s)

0.0024056000002019573


In [89]:
bccb_grav.shape

(400, 200)

### BCCB  Mag

In [90]:
Hxx,Hxy,Hxz,Hyy,Hyz,Hzz = h_bttb_mag(xi,yi,zi,zj,F,h,shape)

In [108]:
s = time()
bccb_mag = ones_cev_mag(Hxx,Hxy,Hxz,Hyy,Hyz,Hzz,shape,N,F,h)
w = time()
print(w-s)

0.012353000000075554


## BCCB Grav - construction with no for

In [92]:
BTTB_grav2 = bttb(xi,yi,zi,zj)
BTTB_matrix = BTTB_grav2.reshape(shape[0],shape[1])

s = time()
bccb_grav_no_for = np.zeros((shape[0]*2,shape[1]*2))
bccb_grav_no_for[:shape[0],:shape[1]] = BTTB_matrix
bccb_grav_no_for[:shape[0],shape[1]+1:] = BTTB_matrix[:,1:shape[1]][:,::-1]
bccb_grav_no_for[shape[0]+1:,:] = bccb_grav_no_for[1:shape[0],:][::-1]
w = time()
print(w-s)

0.0004923999999846274


In [93]:
np.allclose(bccb_grav,bccb_grav_no_for, atol=1e-12)

True

In [94]:
np.allclose(np.fft.fft2(bccb_grav),np.fft.fft2(bccb_grav_no_for), atol=1e-15)

True

## BCCB Mag - construction with no for

In [95]:
Hxx,Hxy,Hxz,Hyy,Hyz,Hzz = h_bttb_mag(xi,yi,zi,zj,F,h,shape)

In [109]:
Hxx_matrix = Hxx.reshape(shape[0],shape[1])
Hxy_matrix = Hxy.reshape(shape[0],shape[1])
Hxz_matrix = Hxz.reshape(shape[0],shape[1])
Hyy_matrix = Hyy.reshape(shape[0],shape[1])
Hyz_matrix = Hyz.reshape(shape[0],shape[1])
Hzz_matrix = -Hxx_matrix-Hyy_matrix

s = time()
bccb_xx = np.zeros((shape[0]*2,shape[1]*2))
bccb_xx[:shape[0],:shape[1]] = Hxx_matrix
bccb_xx[:shape[0],shape[1]+1:] = Hxx_matrix[:,1:shape[1]][:,::-1]
bccb_xx[shape[0]+1:,:] = bccb_xx[1:shape[0],:][::-1]

bccb_xy = np.zeros((shape[0]*2,shape[1]*2))
bccb_xy[:shape[0],:shape[1]] = Hxy_matrix
bccb_xy[:shape[0],shape[1]+1:] = -Hxy_matrix[:,1:shape[1]][:,::-1]
bccb_xy[shape[0]+1:,:] = -bccb_xy[1:shape[0],:][::-1]

bccb_xz = np.zeros((shape[0]*2,shape[1]*2))
bccb_xz[:shape[0],:shape[1]] = Hxz_matrix
bccb_xz[:shape[0],shape[1]+1:] = Hxz_matrix[:,1:shape[1]][:,::-1]
bccb_xz[shape[0]+1:,:] = -bccb_xz[1:shape[0],:][::-1]

bccb_yy = np.zeros((shape[0]*2,shape[1]*2))
bccb_yy[:shape[0],:shape[1]] = Hyy_matrix
bccb_yy[:shape[0],shape[1]+1:] = Hyy_matrix[:,1:shape[1]][:,::-1]
bccb_yy[shape[0]+1:,:] = bccb_yy[1:shape[0],:][::-1]

bccb_yz = np.zeros((shape[0]*2,shape[1]*2))
bccb_yz[:shape[0],:shape[1]] = Hyz_matrix
bccb_yz[:shape[0],shape[1]+1:] = -Hyz_matrix[:,1:shape[1]][:,::-1]
bccb_yz[shape[0]+1:,:] = bccb_yz[1:shape[0],:][::-1]

bccb_zz = np.zeros((shape[0]*2,shape[1]*2))
bccb_zz[:shape[0],:shape[1]] = Hzz_matrix
bccb_zz[:shape[0],shape[1]+1:] = Hzz_matrix[:,1:shape[1]][:,::-1]
bccb_zz[shape[0]+1:,:] = bccb_zz[1:shape[0],:][::-1]

bccb_mag_no_for = 100*((F[0]*bccb_xx+F[1]*bccb_xy+F[2]*bccb_xz)*h[0] + (F[0]*
                        bccb_xy+F[1]*bccb_yy+F[2]*bccb_yz)*h[1] + (F[0]*bccb_xz+F[1]*
                        bccb_yz+F[2]*bccb_zz)*h[2])
w = time()
print(w-s)

0.004857200000060402


In [118]:
Hxx_matrix = Hxx.reshape(shape[0],shape[1])
Hxy_matrix = Hxy.reshape(shape[0],shape[1])
Hxz_matrix = Hxz.reshape(shape[0],shape[1])
Hyy_matrix = Hyy.reshape(shape[0],shape[1])
Hyz_matrix = Hyz.reshape(shape[0],shape[1])

s = time()
bccb_xx = np.zeros((shape[0]*2,shape[1]*2))
bccb_xx[:shape[0],:shape[1]] = Hxx_matrix
bccb_xx[:shape[0],shape[1]+1:] = Hxx_matrix[:,1:shape[1]][:,::-1]
bccb_xx[shape[0]+1:,:] = bccb_xx[1:shape[0],:][::-1]

bccb_xy = np.zeros((shape[0]*2,shape[1]*2))
bccb_xy[:shape[0],:shape[1]] = Hxy_matrix
bccb_xy[:shape[0],shape[1]+1:] = -Hxy_matrix[:,1:shape[1]][:,::-1]
bccb_xy[shape[0]+1:,:] = -bccb_xy[1:shape[0],:][::-1]

bccb_xz = np.zeros((shape[0]*2,shape[1]*2))
bccb_xz[:shape[0],:shape[1]] = Hxz_matrix
bccb_xz[:shape[0],shape[1]+1:] = Hxz_matrix[:,1:shape[1]][:,::-1]
bccb_xz[shape[0]+1:,:] = -bccb_xz[1:shape[0],:][::-1]

bccb_yy = np.zeros((shape[0]*2,shape[1]*2))
bccb_yy[:shape[0],:shape[1]] = Hyy_matrix
bccb_yy[:shape[0],shape[1]+1:] = Hyy_matrix[:,1:shape[1]][:,::-1]
bccb_yy[shape[0]+1:,:] = bccb_yy[1:shape[0],:][::-1]

bccb_yz = np.zeros((shape[0]*2,shape[1]*2))
bccb_yz[:shape[0],:shape[1]] = Hyz_matrix
bccb_yz[:shape[0],shape[1]+1:] = -Hyz_matrix[:,1:shape[1]][:,::-1]
bccb_yz[shape[0]+1:,:] = bccb_yz[1:shape[0],:][::-1]

bccb_zz = -bccb_xx-bccb_yy

bccb_mag_no_for = 100*((F[0]*bccb_xx+F[1]*bccb_xy+F[2]*bccb_xz)*h[0] + (F[0]*
                        bccb_xy+F[1]*bccb_yy+F[2]*bccb_yz)*h[1] + (F[0]*bccb_xz+F[1]*
                        bccb_yz+F[2]*bccb_zz)*h[2])
w = time()
print(w-s)

0.00491720000013629


In [111]:
np.allclose(bccb_mag,bccb_mag_no_for, atol=1e-12)

True

In [112]:
np.allclose(np.fft.fft2(bccb_mag),np.fft.fft2(bccb_mag_no_for), atol=1e-15)

True

## Tests

In [145]:
Nx = 4
Ny = 3
a = np.array([[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12]])
b = a.reshape(Nx,Ny)
b

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]])

In [146]:
c = np.zeros((Nx*2,Ny*2))
c

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [147]:
c[:Nx,:Ny] = b
c

array([[ 1.,  2.,  3.,  0.,  0.,  0.],
       [ 4.,  5.,  6.,  0.,  0.,  0.],
       [ 7.,  8.,  9.,  0.,  0.,  0.],
       [10., 11., 12.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [148]:
c[:Nx,Ny+1:] = b[:,1:Ny][:,::-1]
c

array([[ 1.,  2.,  3.,  0.,  3.,  2.],
       [ 4.,  5.,  6.,  0.,  6.,  5.],
       [ 7.,  8.,  9.,  0.,  9.,  8.],
       [10., 11., 12.,  0., 12., 11.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [149]:
c[Nx+1:,:] = c[1:Nx,:][::-1]
c

array([[ 1.,  2.,  3.,  0.,  3.,  2.],
       [ 4.,  5.,  6.,  0.,  6.,  5.],
       [ 7.,  8.,  9.,  0.,  9.,  8.],
       [10., 11., 12.,  0., 12., 11.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [10., 11., 12.,  0., 12., 11.],
       [ 7.,  8.,  9.,  0.,  9.,  8.],
       [ 4.,  5.,  6.,  0.,  6.,  5.]])

In [108]:
c[1:Nx,:][::-1]

array([[5., 6., 0., 6.],
       [3., 4., 0., 4.]])

In [96]:
z = np.array([[1, 2],[3, 4]])
z[:,::-1]

array([[2, 1],
       [4, 3]])